In [0]:
import torch
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
import copy
import random
from collections import Counter
import nltk
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Train data & Test data setting
# Load data set
positive_data = open('/content/gdrive/My Drive/rt-polaritydata/rt-polarity.pos.txt', mode='rb').read()
negative_data = open('/content/gdrive/My Drive/rt-polaritydata/rt-polarity.neg.txt', mode='rb').read()

# Make entire words embedding vectors
# 1. convert bytes to utf-8
decoded_positive_data = positive_data.decode("utf-8", errors="replace")
decoded_negative_data = negative_data.decode("utf-8", errors="replace")

# 2. split by sentences with nltk
pre_positive_sentences =  list(set([
                      sentence for sentence in nltk.sent_tokenize(decoded_positive_data)
                      ]))

pre_negative_sentences = list(set([
                      sentence for sentence in nltk.sent_tokenize(decoded_negative_data)
                      ]))

positive_sentences = [item for item in pre_positive_sentences if item not in pre_negative_sentences]
negative_sentences = [item for item in pre_negative_sentences if item not in pre_positive_sentences]

# 2-1. give a class label to each sentence group
positive_classes = [1 for i in range(len(positive_sentences))]
negative_classes = [0 for i in range(len(negative_sentences))]

# combine sentences and classses
all_sentences = positive_sentences + negative_sentences
all_classes = positive_classes + negative_classes

# 3. give indices/class label to every sentences
# sentence index to sentence & sentence to sentence index
si2s = {}
s2si = {}
for index, sentence in enumerate(all_sentences):
    si2s[index] = sentence
    s2si[sentence] = index

# sentence index to class label
si2c = {}
for index, class_label in enumerate(all_classes):
    si2c[index] = class_label

# 4.get all words list
# 4-1. get max numbers of words in each one sentence
word_list = []
# sentence_length = []
stopwords = [',', '.', '(', ')', '-', '--', '[', ']', '`', '\'', '``', '\"' ]
for sentence in all_sentences:
    sentence_words = [
                      word for word in nltk.word_tokenize(sentence)
                      if word not in stopwords
                      ]
    word_list += sentence_words
    # sentence_length.append(len(sentence_words))

word_list += ['<PAD>']

# number of all sentences/words
num_sentences = len(all_sentences)
num_words = len(word_list)
# max_sentence = max(sentence_length)

# 5. give indices to every words
wi2w = {}
w2wi = {}
for index, word in enumerate(word_list):
    wi2w[index] = word
    w2wi[word] = index

# 6. give word indices to each sentence index
si2wi = {}
wi2si = {}
for sentence in all_sentences:
    sent_index = s2si[sentence]
    sentence_words = [
                      word for word in nltk.word_tokenize(sentence)
                      if word not in stopwords
                      ]
    sentence_word_indices = [w2wi[word] for word in sentence_words]
    si2wi[sent_index] = sentence_word_indices

In [0]:
# split Training set and Test set with 9:1
# 정확한 비율은 아니나 편의를 위해 10000:나머지 로 나눈다.
sample_list = [i for i in range(num_sentences)]
testing_set = random.sample(sample_list, (num_sentences-10000))
training_set = [index for index in sample_list if index not in testing_set]

# shuffle data set
random.shuffle(training_set)
random.shuffle(testing_set)

num_training_set = len(training_set)
num_testing_set = len(testing_set)

In [0]:
args = {
    'vocab_num': num_words,
    'dimension': 300,
    'class_num': 2,
    'filter_num': 100,
    'filter_size': [3,4,5],
    'dropout': 0.5,
    'epoch': 10,
    'learning_rate': 0.001,
    'batch_size': 50,
    'hidden_layer': 100,
    'padding_index': w2wi['<PAD>'],
    'mode': 'rand'
}
        

In [0]:
# Google News vector 사용위한 function
def get_vector():
    vectors = []
    word2vec = KeyedVectors.load_word2vec_format(
                '/content/gdrive/My Drive/rt-polaritydata/GoogleNews-vectors-negative300.bin',
                binary=True)
    for i in range(num_words):
        word = wi2w[i]
        if word in word2vec.vocab:
            vectors.append(torch.from_numpy(word2vec[word]))
        else:
            vectors.append(torch.FloatTensor(args['dimension']).uniform_(-0.01, 0.01))

    return torch.stack(vectors)

In [0]:
# CNN model
class CNN(nn.Module):
    
    def __init__(self, args):
        super(CNN, self).__init__()
        self.args = args
        
        # mode = 'rand'일때
        # 전체 vocab의 수(words의 수)
        vocab_num = args['vocab_num']
        # embedding vector의 feature dimension 수
        dimension = args['dimension']
        # class number
        class_num = args['class_num']
        # in channels
        in_channel = 1
        # hidden layer size
        hidden_layer = args['hidden_layer']
        # filter num
        filter_num = args['filter_num']
        # filter size default = [3,4,5]
        filter_size = args['filter_size']
        # dropout rate
        dropout = args['dropout']
        # padding index
        self.padding_index = args['padding_index']

        if args['mode'] == 'rand':
            self.embedding = nn.Embedding(vocab_num, dimension, padding_idx=self.padding_index)

        # rand가 아니면 embedding vector는 pre-trained 구글 벡터를 가져오기
        google_vectors = get_vector()
        if args['mode'] != 'rand':
            self.embedding = nn.Embedding.from_pretrained(google_vectors, freeze=False)
            
        # static일 경우는 vector update없이
        if args['mode'] == 'static':
            self.embedding.freeze = True

        # multi-channel은 하나의 embedding vector를 더 사용
        # 둘 다 pre-trained를 사용하지만, 하나만 학습되고 하나는 static
        # embedding2가 static하도록 설정
        if args['mode'] == 'multichannel':
            self.embedding2 = nn.Embedding.from_pretrained(google_vectors)
            in_channel = 2

        self.convs_1ist = nn.ModuleList(
            [nn.Conv2d(in_channel, filter_num, (size, dimension)) for size in filter_size]
            )
        self.linear = nn.Sequential(
            nn.Linear(len(filter_size)*filter_num, hidden_layer),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_layer, class_num),

        )
        # self.dropout = nn.Dropout(args['dropout'])
        # self.fully_connected = nn.Linear(len(filter_size)*filter_num, class_num)

    def forward(self, x):
        # multichannel의 경우 concatenate하여 두 vector를 쌓아올려준다
        if args['mode'] == 'multichannel':
            non_static_embedding = self.embedding(x)
            static_embedding = self.embedding2(x)
            x = torch.stack([non_static_embedding, static_embedding], dim=1)
        else :
            x = self.embedding(x)
            x = x.unsqueeze(1)    
        

        x_relu = [F.relu(conv(x)).squeeze(3) for conv in self.convs_1ist]
        x_pool = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x_relu]

        x_concat = torch.cat(x_pool, 1)
        output = self.linear(x_concat)
        return output

In [0]:
def train(args):
    epochs = args['epoch']
    learning_rate = args['learning_rate']
    batch_size = args['batch_size']

    model = CNN(args)
    optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss(reduction='sum')

    loss_list = []

    # Training session
    for epoch in range(0, epochs):
        model.train()

        for start_index, end_index in zip(
            range(0, num_training_set, batch_size),
            range(batch_size, num_training_set+1, batch_size)):
            train_list = []
            class_list = []
            pad = w2wi['<PAD>']
            max_len = 50
            train_loss = 0

            for sentence_index in training_set[start_index: end_index]:
                length = len(si2wi[sentence_index])
                if length < max_len:
                    si2wi[sentence_index] += [pad] * (max_len - length)
                elif length > max_len:
                    si2wi[sentence_index] = si2wi[sentence_index][:max_len]

                train_list.append(
                    torch.unsqueeze(Variable(torch.LongTensor(si2wi[sentence_index])), 0)
                    )
                class_list.append(si2c[sentence_index])

            scores = model(torch.squeeze(torch.stack(train_list, 1)))
            var_class = Variable(torch.LongTensor(class_list))

            predict = F.softmax(scores, dim=1).argmax(dim=1)# 

            accuracy = torch.sum(predict == var_class).item() / batch_size

            loss = criterion(scores, var_class)

            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f'Epoch: {epoch}\nloss: {train_loss/batch_size}, accuracy: {accuracy}')

        loss_list.append(train_loss/batch_size)

    # Test session
    model.eval()
    pad = w2wi['<PAD>']
    max_len = 50
    test_list = []
    index_list = []
    for sentence_index in testing_set:
        length = len(si2wi[sentence_index])
        if length < max_len:
            si2wi[sentence_index] += [pad] * (max_len - length)
        elif length > max_len:
            si2wi[sentence_index] = si2wi[sentence_index][:max_len]

        test_list.append(
            torch.unsqueeze(Variable(torch.LongTensor(si2wi[sentence_index])), 0)
            )
        index_list.append(si2c[sentence_index])

    test_scores = model(torch.squeeze(torch.stack(test_list, 1)))
    test_class = Variable(torch.LongTensor(index_list))

    test_predict = F.softmax(test_scores, dim=1).argmax(dim = 1)
    test_accuracy = torch.sum(test_predict == test_class).item() / num_testing_set
    loss = criterion(test_scores, test_class)

    print(f'Test Result\nTest Loss: {loss.item()/num_testing_set}, Test Accuracy: {test_accuracy}')

In [0]:
train(args)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Epoch: 0
loss: 0.5757014083862305, accuracy: 0.7
Epoch: 1
loss: 0.3170578765869141, accuracy: 0.84
Epoch: 2
loss: 0.03739868640899658, accuracy: 0.98
Epoch: 3
loss: 0.02394737720489502, accuracy: 1.0
Epoch: 4
loss: 0.004188187420368195, accuracy: 1.0
Epoch: 5
loss: 0.002179316133260727, accuracy: 1.0
Epoch: 6
loss: 0.0029251980781555176, accuracy: 1.0
Epoch: 7
loss: 7.993518374860287e-05, accuracy: 1.0
Epoch: 8
loss: 0.0004614561423659325, accuracy: 1.0
Epoch: 9
loss: 0.0154403817653656, accuracy: 1.0
Test Result
Test Loss: 2.0243234536082473, Test Accuracy: 0.7439862542955327


In [0]:
args['mode'] = 'static'
train(args)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Epoch: 0
loss: 0.5616396713256836, accuracy: 0.7
Epoch: 1
loss: 0.3522276306152344, accuracy: 0.86
Epoch: 2
loss: 0.07755455493927002, accuracy: 0.98
Epoch: 3
loss: 0.008515973687171935, accuracy: 1.0
Epoch: 4
loss: 0.002447342425584793, accuracy: 1.0
Epoch: 5
loss: 0.0019356055557727813, accuracy: 1.0
Epoch: 6
loss: 0.003267463743686676, accuracy: 1.0
Epoch: 7
loss: 0.09863566398620606, accuracy: 0.96
Epoch: 8
loss: 0.07304125308990478, accuracy: 0.98
Epoch: 9
loss: 0.0001586354523897171, accuracy: 1.0
Test Result
Test Loss: 2.2795057788337627, Test Accuracy: 0.7173539518900344


In [0]:
args['mode'] = 'non-static'
train(args)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Epoch: 0
loss: 0.49863475799560547, accuracy: 0.7
Epoch: 1
loss: 0.3798704147338867, accuracy: 0.82
Epoch: 2
loss: 0.04664958953857422, accuracy: 0.98
Epoch: 3
loss: 0.010416384935379028, accuracy: 1.0
Epoch: 4
loss: 0.008935202956199645, accuracy: 1.0
Epoch: 5
loss: 0.0006413940340280533, accuracy: 1.0
Epoch: 6
loss: 0.004944433271884918, accuracy: 1.0
Epoch: 7
loss: 0.002360856980085373, accuracy: 1.0
Epoch: 8
loss: 0.00354162335395813, accuracy: 1.0
Epoch: 9
loss: 0.07173749446868896, accuracy: 0.96
Test Result
Test Loss: 2.0952786055627146, Test Accuracy: 0.7044673539518901


In [0]:
args['mode'] = 'multichannel'
train(args)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Epoch: 0
loss: 0.524925308227539, accuracy: 0.7
Epoch: 1
loss: 0.4020498275756836, accuracy: 0.84
Epoch: 2
loss: 0.08921781539916993, accuracy: 0.98
Epoch: 3
loss: 0.006928830742835999, accuracy: 1.0
Epoch: 4
loss: 0.007732195258140564, accuracy: 1.0
Epoch: 5
loss: 0.001774752289056778, accuracy: 1.0
Epoch: 6
loss: 0.013980120420455933, accuracy: 1.0
Epoch: 7
loss: 0.00020490977913141252, accuracy: 1.0
Epoch: 8
loss: 0.019507091045379638, accuracy: 0.98
Epoch: 9
loss: 0.02264051914215088, accuracy: 0.98
Test Result
Test Loss: 2.0032958984375, Test Accuracy: 0.7289518900343642
